In [7]:
import torch
import torchvision
import imutils
import cv2
import cvzone
import warnings
import mediapipe as mp

from torch import nn
from copy import deepcopy
from PIL import Image as Imag
from torchvision import datasets, models, transforms
from cvzone.HandTrackingModule import HandDetector
warnings.filterwarnings("ignore", category=UserWarning)

print("PyTorch Version; ",torch.__version__)
print("Torchvision Version: ",torchvision.__version__)

PyTorch Version;  2.1.2
Torchvision Version:  0.16.2


In [2]:
def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False
def initialize_model(model_name, num_classes, feature_extract, use_pretrained=True):
    model_ft = None
    input_size = 0
    if model_name == "DenseNet":
        """ Densenet
        """
        model_ft = models.densenet121(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.classifier.in_features
        model_ft.classifier = nn.Linear(num_ftrs, num_classes)
        input_size = 224
    else:
        print("Invalid model name, exiting...")
        exit()
    return model_ft, input_size
def probabilidad(frame):
    img = frame
    img = imutils.resize(img, width=600)
    img = cv2.cvtColor(deepcopy(img), cv2.COLOR_BGR2RGB)
    image_pil = Imag.fromarray(img)
    input_size = 224
    data_transform = transforms.Compose([
      transforms.Resize(input_size),
      transforms.CenterCrop(input_size),
      transforms.ToTensor(),
      transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
    img = data_transform(image_pil)
    img = img.unsqueeze(0).to(device)
    out = model_ft(img).cpu().detach().tolist()[0]
    A_L, A_R, B_L, B_R, C_L, C_R, D_L, D_R, E_L, E_R, F_L, F_R, G_L, G_R, H_L, H_R, I_L, I_R, J_L, J_R, K_L, K_R, L_L, L_R, M_L, M_R, N_L, N_R, O_L, O_R, P_L, P_R, Q_L, Q_R, R_L, R_R, S_L, S_R, T_L, T_R, U_L, U_R, V_L, V_R, W_L, W_R, X_L, X_R, Y_L, Y_R, Z_L, Z_R = torch.nn.functional.softmax(torch.tensor(out)).tolist()
    symbols = {A_L:"A", A_R:"A", B_L:"B", B_R:"B", C_L:"C", C_R:"C", D_L:"D", D_R:"D", E_L:"E", E_R:"E", F_L:"F", F_R:"F", G_L:"G", G_R:"G", H_L:"H", H_R:"H", I_L:"I", I_R:"I", J_L:"J", J_R:"J", K_L:"K", K_R:"K", L_L:"L", L_R:"L", M_L:"M", M_R:"M", N_L:"N", N_R:"N", O_L:"O", O_R:"O", P_L:"P", P_R:"P", Q_L:"Q", Q_R:"Q", R_L:"R", R_R:"R", S_L:"S", S_R:"S", T_L:"T", T_R:"T", U_L:"U", U_R:"U", V_L:"V", V_R:"V", W_L:"W", W_R:"W", X_L:"X", X_R:"X", Y_L:"Y", Y_R:"Y", Z_L:"Z", Z_R:"Z"}
    maxim = max(torch.nn.functional.softmax(torch.tensor(out)).tolist())
    labels = str(symbols.get(maxim))
    return maxim, labels

In [3]:
model_ft, input_size = initialize_model("DenseNet", 52, False, use_pretrained=True)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model_ft.eval()
model_ft.to(device)
model_ft.load_state_dict(torch.load('D:/CPU/best.pt')['model_state_dict'])

<All keys matched successfully>

In [6]:
print("[INFO] starting video stream...")
cap = cv2.VideoCapture(0, cv2.CAP_DSHOW)
#cap = cv2.VideoCapture('D:/Lenguaje de señas/A.mp4')
detector = HandDetector(detectionCon=0.8,maxHands=1)
i=0
while True:
    success, img = cap.read()
    hands = detector.findHands(img, draw=False)
    if hands:
        hand1 = hands[0]
        if hand1 and "bbox" in hand1[1]:
            bbox = hand1[0]['bbox']
            x, y, ancho, alto = bbox
            x1, y1, x2, y2 = x, y, ancho, alto
            x1, y1 = max(x1,0), max(y1,0)
            
            img = cvzone.cornerRect(img, (x1-20, y1-20, x2+40, y2+40))
            
            maximo, etiqueta = probabilidad(img)
            maximo = str(maximo*100)+str('%')
            
            t1 = x1 - 25, y1 - 70
            t2 = x1 - 25, y1 - 40
            
            cv2.putText(img, etiqueta, (int(t1[0]), int(t1[1])),cv2.FONT_HERSHEY_SIMPLEX, 1,(0,255,0),2)
            cv2.putText(img, maximo, (int(t2[0]), int(t2[1])),cv2.FONT_HERSHEY_SIMPLEX, 1,(0,255,0),2)
    
    cv2.imshow("Image", img)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
        
cap.release()
cv2.destroyAllWindows()
#pip install mediapipe

[INFO] starting video stream...


NameError: name 'hand' is not defined

In [ ]:
pip install cv2

In [8]:
import cv2
def indice_camara():
    index = -2
    arr = []
    i = 10
    while i > 0:
        cap = cv2.VideoCapture(index)
        if cap.read()[0]:
            arr.append(index)
            cap.release()
        index += 1
        i -= 1
    return arr
print(indice_camara())
        

[0]


In [9]:
import cv2

cap = cv2.VideoCapture(0);

while(True):
    #Captura frame-by-frame
    ret, frame = cap.read();

    #Muestra el resultado de frame
    cv2.imshow('frame',frame)
    if cv2.waitKey(20) & 0xFF == ord('q'):
        break

#Cuando todo este hecho cierra el programa
cap.release()
cv2.destroyAllWindows()